In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import mbore
import pymoo

%load_ext autoreload
%autoreload 2

### problem setup

In [ ]:
# here we'll optimise the DTLZ2 test problem, from the pymoo package
n_dim = 2
n_obj = 3

problem = pymoo.problems.many.DTLZ2(n_var=n_dim, n_obj=n_obj)

lb, ub = problem.bounds()
f = problem.evaluate # objective function

# its reference point
ref_point = np.array([2.0] * n_obj)

# generate some uniform random samples
n_train = 10

Xtr = np.random.uniform(lb, ub, (n_train, n_dim))
Ytr = f(Xtr)

# save the data (as an example of saving + loading)
filename = "testing_saving_and_loading.npz"
np.savez(filename, Xtr=Xtr, Ytr=Ytr, ref_point=ref_point)

# load back the data
with np.load(filename) as fd:
    Xtr = fd["Xtr"]
    Ytr = fd["Ytr"]
    ref_point = fd["ref_point"]

### settings and setup

In [ ]:
scalariser_name = "HypCont" # "HypI", "DomRank", "HypCont", "ParEGO"
optimisation_budget = 1024 * n_dim # budget for optimising model (acq func equivalent)
gamma = 1/3 # proportion of solutions to include in the *good* class
weight_type = "ei" # "ei" or "pi"

# get a "ranker" (scaleriser) class
scalariser_class = mbore.util.get_ranker(scalariser_name)

# instantiate the class
scalariser = scalariser_class(ref_point)

#### one step of the optimisation

In [ ]:
# scalarise the objective values
_, scalers = scalariser.get_ranks(Ytr, return_scalers=True)

# rescale decision vectors to [0, 1] and declare the associated bounds
xtransformer = mbore.transforms.ZeroOneTransform(Xtr, lb, ub)
sXtr = xtransformer.transform(Xtr)
slb, sub = np.zeros(n_dim), np.ones(n_dim)

# get the model weights and data for the classifier training
# (weights are all equal for "pi", not for "ei")
x, y, z, w, tau = mbore.util.load_classification_data(
    sXtr, scalers, gamma, weight_type=weight_type
)

# define and train the classifier
clf = mbore.classifiers.XGBoostClassifier(Xtr=x, class_labels=z, weights=w)
clf.train()

# instantiate the optimiser
if False:
    # here we use random search and take the argmax (boltzman_sample=False is argmax)
    opt = mbore.optimizers.RandomSearchOptimizer(clf, slb, sub, optimisation_budget, boltzman_sample=False)
else:
    # or we can use CMAES
    opt = mbore.optimizers.CMAESOptimizer(clf, slb, sub, optimisation_budget)

# perform the optimisation of the predictive distribution. note that this point
# is in the scaled space (i.e. [0, 1]^d)
sXnext = opt.get_xnext()
        
# un-scale the suggested value back to the original decision space
Xnext = xtransformer.untransform(sXnext)

# evaluate function
Ynext = problem.evaluate(Xnext)

# store the newly evaluate solution, its fitness and timing
Xtr = np.concatenate((Xtr, Xnext.reshape(1, n_dim)))
Ytr = np.concatenate((Ytr, Ynext.reshape(1, n_obj)))